# Turtle runs - RL with the smoothed Crossentropy Method

In [ ]:
import sys
import numpy as np
from collections import namedtuple
from time import time

In [ ]:
actions = ['N', 'E', 'S', 'W', 'O']

In [ ]:
class Turtle():
    def __init__(self, position, halite):
        self.position = position
        self.halite = halite

In [ ]:
class GameState():
    def __init__(self, game_map, position, halite):
        self.game_map = game_map
        self.position = position
        self.halite = halite
        
    def __eq__(self, other):
        return (self.game_map == other.game_map).all() and self.position == other.position and self.halite == other.halite

    def __contains__(self, key):
        return key in self.numbers


In [ ]:
class SimpleHalite():
    def __init__(self, height, width, start_pos):
        np.random.seed(42)
        self.game_map = np.random.randint(1, 1000, size=(height, width))
        self.game_map[start_pos] = 0
        self.orig_map = self.game_map.copy()
        self.turtle = Turtle(start_pos, 0)
        self.turn = 1
        self.max_turns = 50
        self.halite = 0
        self.base = start_pos
        self.height = height
        self.width = width
    
    def get_state(self):
        game_state = GameState(self.game_map.copy(), self.turtle.position, self.turtle.halite)
        return game_state, self.turn, self.halite, self.turn == self.max_turns
    
    def reset(self):
        self.game_map = self.orig_map.copy()
        self.turtle = Turtle(self.base, 0)
        self.turn = 1
        self.halite = 0
        
    def step(self, action):
        reward = 0
        if action == 'O':
            mined_halite = self.game_map[self.turtle.position] // 4
            self.game_map[self.turtle.position] -= mined_halite
            self.turtle.halite += min(1000, mined_halite)
        else:
            if action == 'N':
                cost_halite = self.game_map[self.turtle.position] // 10
                new_pos = tuple([sum(x) for x in zip(self.turtle.position, (0, 1))])
            elif action == 'E':
                cost_halite = self.game_map[self.turtle.position] // 10
                new_pos = tuple([sum(x) for x in zip(self.turtle.position, (1, 0))])
            elif action == 'S':
                cost_halite = self.game_map[self.turtle.position] // 10
                new_pos = tuple([sum(x) for x in zip(self.turtle.position, (0, -1))])
            elif action == 'W':
                cost_halite = self.game_map[self.turtle.position] // 10
                new_pos = tuple([sum(x) for x in zip(self.turtle.position, (-1, 0))])
            #print(cost_halite, self.turtle.halite)
            if cost_halite <= self.turtle.halite:
                #print("moving turtle to {}".format(new_pos))
                self.turtle = Turtle(new_pos, self.turtle.halite - cost_halite)
            else:
                mined_halite = self.game_map[self.turtle.position] // 4
                self.game_map[self.turtle.position] -= mined_halite
                self.turtle.halite += min(1000, mined_halite)                
        self.turtle.position = (self.turtle.position[0] % self.width, 
                                self.turtle.position[1] % self.height)
        if self.turtle.position == self.base:
            self.halite += self.turtle.halite
            reward = self.turtle.halite
            self.turtle.halite = 0
        self.turn += 1
        return self.get_state()

In [ ]:
def policy(game_state, turn, policy_mapping=None):
    if policy_mapping is not None and (game_state.position, game_state.halite) in policy_mapping:
        for map_action in policy_mapping[(game_state.position, game_state.halite)]:
            if (map_action[0] == game_state.game_map).all():
                return np.random.choice(actions, p=map_action[1])
    return np.random.choice(actions)

Needs to be a tuple rather than a list as start_pos

In [ ]:
game = SimpleHalite(5, 5, (2, 2))

In [ ]:
N = 100
M = 5
alpha = 0.5

In [ ]:
def add_to_policy_mapping(policy_mapping, game_map, position, halite, action_probs):
    if (position, halite) not in policy_mapping:
            policy_mapping[(position, halite)] = []
    policy_mapping[(position, halite)].append((game_map, action_probs))
    return

In [ ]:
import warnings
warnings.filterwarnings("error")

In [ ]:
class ArrayWrapper():
    def __init__(self):
        self.els = []
    
    def append(self, obj):
        self.els.append(obj)
        
    def __contains__(self, obj):
        for el in self.els:
            if (el == obj).all():
                return True
        return False

In [ ]:
policy_mapping = {}
for j in range(M):
    now = time()
    state_action_maps = []
    final_rewards = []
    for i in range(N):
        state_action_map = []
        game.reset()
        turn_count = 0
        game_state, turn, reward, done = game.get_state()
        while not done:
            action = policy(game_state, turn, policy_mapping)
            state_action_map.append((game_state, action))
            turn_count += 1
            game_state, turn, reward, done = game.step(action)
        state_action_maps.append(state_action_map)
        final_rewards.append(reward)
    print(j, np.mean(final_rewards))
    elite_games = np.argsort(final_rewards)[N//5:]
    maps_to_keep = np.array(state_action_maps)[elite_games]
    full_map = {}
    for sam in maps_to_keep:
        for sap in sam:
            if (sap[0].position, sap[0].halite) not in full_map:
                full_map[(sap[0].position, sap[0].halite)] = []
            full_map[(sap[0].position, sap[0].halite)].append((sap[0].game_map, sap[1]))
    print("Entries in policy mapping:", sum(len(x) for x in policy_mapping.values()))
    print("Number of keys:", len(policy_mapping.keys()))
    updated, not_updated = 0, 0
    for ((position, halite), saps) in full_map.items():
        done_policies = ArrayWrapper()
        for (state_map, action) in saps:
            if state_map not in done_policies:
                done_policies.append(state_map)
                subset = [a for (s, a) in saps if (s == state_map).all()]
                action_probs = []
                action_probs_raw = []
                for action in actions:
                    action_probs.append((subset.count(action) + 1) / (len(subset) + 5))
                    action_probs_raw.append(subset.count(action) / len(subset))
                if (position, halite) in policy_mapping: 
                    for ix, (game_map, probs) in enumerate(policy_mapping[position, halite]):
                        if (game_map == state_map).all():
                            policy_mapping[(position, halite)][ix] = \
                                (state_map, alpha * np.array(action_probs_raw) + (1 - alpha) * np.array(probs))
                            updated += 1
                            break
                    not_updated += 1
                    add_to_policy_mapping(policy_mapping, state_map, position, halite, action_probs)
                else:
                    not_updated += 1
                    add_to_policy_mapping(policy_mapping, state_map, position, halite, action_probs)
    print("Updated: {}, not updated: {}".format(updated, not_updated))
    print("Iteration took {:.0f} seconds".format(time() - now))

In [ ]:
[x for x in policy_mapping if max(x[1]) > 0.5]

We can now simulate the game loop. We now need to 
- fix the game map (otherwise the state mappong will become too unwieldy) - DONE
- keep track of state-action pairs (memory intensive, but maybe we can manage) - DONE
- use state-action pairs from elite runs to update policy - DONE
- add smoothing - DONE
- improve policy mapping data structure to keep turn times roughly constant - DONE